In [1]:
#기본적인 Library 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler

#데이터분리
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV

#모델링
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#성과 평가 지표
from sklearn.metrics import accuracy_score, mean_squared_error


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pycaret[full]

In [ ]:
!pip install Jinja2

In [ ]:
!pip install markupsafe==2.0.1

In [2]:
# import pycaret
import pycaret
import jinja2
from pycaret.classification import *

In [3]:
path = '/content/drive/MyDrive/colab/2022/kubig/compet/data/'

train = pd.read_csv(path + './train.csv', index_col=0)
test = pd.read_csv(path + './test_x.csv', index_col=0)
submission = pd.read_csv(path + './sample_submission.csv', index_col=0)

#voted = 0 , unvoted = 1 으로 바꿔주기
train.voted = train.voted - 1

In [4]:
# log transform train
Answers_time_only = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE', 
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE', 
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE']

loged_Answer_time = train[Answers_time_only].copy()
loged_Answer_time[Answers_time_only] = np.log1p(train[Answers_time_only])
train[Answers_time_only] = loged_Answer_time

In [5]:
# log transform test
log_Answer_time_t = test[Answers_time_only].copy()
log_Answer_time_t[Answers_time_only] = np.log1p(test[Answers_time_only])
test[Answers_time_only] = log_Answer_time_t

In [6]:
# outlier(familysize)
train = train.drop(train[train.familysize >= 20].index)

# 무응답자 대체
### 중간값으로

In [7]:
print(train['education'].value_counts())
print(train['engnat'].value_counts())
print(train['hand'].value_counts())
print(train['married'].value_counts())
print(train['urban'].value_counts())

2    16826
3    15226
4     7853
1     5088
0      527
Name: education, dtype: int64
1    33013
2    12430
0       77
Name: engnat, dtype: int64
1    39049
2     4691
3     1619
0      161
Name: hand, dtype: int64
1    31543
2    10057
3     3827
0       93
Name: married, dtype: int64
2    18529
3    17764
1     8905
0      322
Name: urban, dtype: int64


In [8]:
from collections import Counter

In [9]:
## train
# engnat -> drop ?
# train = train.drop(train[train.familysize >= 20].index)

no_ans_col = ['education', 'hand','married' ,'urban' ]

for i in no_ans_col:
  print(i)
  print(Counter(train[i]))
  train[i] = train[i].replace(0, np.median(train[i]))
  print(Counter(train[i]))
  print()

education
Counter({2: 16826, 3: 15226, 4: 7853, 1: 5088, 0: 527})
Counter({2: 16826, 3: 15753, 4: 7853, 1: 5088})

hand
Counter({1: 39049, 2: 4691, 3: 1619, 0: 161})
Counter({1: 39210, 2: 4691, 3: 1619})

married
Counter({1: 31543, 2: 10057, 3: 3827, 0: 93})
Counter({1: 31636, 2: 10057, 3: 3827})

urban
Counter({2: 18529, 3: 17764, 1: 8905, 0: 322})
Counter({2: 18851, 3: 17764, 1: 8905})



In [10]:
## test 
no_ans_col = ['education', 'hand','married' ,'urban' ]

for i in no_ans_col:
  print(i)
  print(Counter(test[i]))
  test[i] = test[i].replace(0, np.median(test[i]))
  print(Counter(test[i]))
  print()

education
Counter({2: 4236, 3: 3798, 4: 1963, 1: 1264, 0: 122})
Counter({2: 4236, 3: 3920, 4: 1963, 1: 1264})

hand
Counter({1: 9818, 2: 1155, 3: 370, 0: 40})
Counter({1: 9858, 2: 1155, 3: 370})

married
Counter({1: 7928, 2: 2469, 3: 958, 0: 28})
Counter({1: 7956, 2: 2469, 3: 958})

urban
Counter({2: 4551, 3: 4439, 1: 2299, 0: 94})
Counter({2: 4645, 3: 4439, 1: 2299})



In [11]:
clf = setup(data = train, target = 'voted')

,Description,Value
0,session_id,6474
1,Target,voted
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(45520, 77)"
5,Missing Values,False
6,Numeric Features,40
7,Categorical Features,36
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='voted',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strate...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

In [12]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.6921,0.7637,0.6531,0.7498,0.6980,0.3870,0.3909,25.075
gbc,Gradient Boosting Classifier,0.6912,0.7625,0.6344,0.7597,0.6913,0.3873,0.3938,22.493
lightgbm,Light Gradient Boosting Machine,0.6906,0.7616,0.6363,0.7573,0.6915,0.3858,0.3918,1.387
lda,Linear Discriminant Analysis,0.6914,0.7605,0.6544,0.7479,0.6980,0.3853,0.3889,1.480
lr,Logistic Regression,0.6910,0.7598,0.6599,0.7442,0.6995,0.3839,0.3869,8.334
et,Extra Trees Classifier,0.6880,0.7586,0.6360,0.7533,0.6896,0.3805,0.3861,9.463
ada,Ada Boost Classifier,0.6890,0.7566,0.6460,0.7491,0.6937,0.3814,0.3858,4.387
rf,Random Forest Classifier,0.6856,0.7523,0.6277,0.7542,0.6851,0.3764,0.3828,9.574
xgboost,Extreme Gradient Boosting,0.6741,0.7458,0.6563,0.7208,0.6870,0.3487,0.3504,22.214
nb,Naive Bayes,0.6585,0.7205,0.6718,0.6997,0.6786,0.3133,0.3204,0.093


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 2
INFO:logs:[<catboost.core.CatBoostClassifier object at 0x7f495c3822d0>, GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=6474, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learnin

In [13]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7006,0.7705,0.6432,0.7695,0.7008,0.4060,0.4125
1,0.7096,0.7755,0.6435,0.7848,0.7072,0.4246,0.4329
2,0.6923,0.7628,0.6326,0.7623,0.6914,0.3898,0.3966
3,0.6893,0.7570,0.6360,0.7552,0.6905,0.3831,0.3888
4,0.6842,0.7631,0.6430,0.7431,0.6894,0.3717,0.3757
Mean,0.6952,0.7658,0.6397,0.7630,0.6959,0.3950,0.4013
Std,0.0089,0.0065,0.0045,0.0140,0.0069,0.0185,0.0198


INFO:logs:create_model_container: 17
INFO:logs:master_model_container: 17
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f495bfd4610>),
                             ('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                 

In [14]:
pred_holdout = predict_model(blended)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f495bfd4610>),
                             ('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6974,0.7698,0.6438,0.7695,0.7011,0.3997,0.4062


In [15]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f495bfd4610>),
                             ('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_spl

In [17]:
predictions = predict_model(final_model, data = test)

submission['voted'] = predictions['Label']

submission.to_csv(path+'/submission_proba(log_out_change).csv')

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f495d8f81d0>),
                             ('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split

In [18]:
submission['voted'] = predictions['Label']

submission = submission + 1
submission.to_csv(path+'/submission_proba(log_out_change).csv')